# Round of 16 draw for 2017-2018 season

Nyon - 11 December 2017, 12:00 CET

Draw procedure:

* Two seeding pots will be formed: one consisting of group winners and the other of runners-up.
* No team can play a club from their group or any side from their own association.
* Seeded group winners will be away in the round of 16 first legs and at home in the return matches.

|         **POT Winners**        |      **POT Runners-up**       |
| :----------------------------: | :---------------------------: |
| Manchester United (A, England) | Basel (A, Switzerland)        |
| PSG (B, France)                | Bayer Munchen (B, Germany)    |
| Roma (C, Italy)                | Chelsea (C, England)          |
| Barcelona (D, Spain)           | Juventus (D, Italy)           |
| Liverpool (E, England)         | Sevilla (E, Spain)            |
| Manchester City (F, England)   | Shakhtar Donetsk (F, Ukraine) |
| Besiktas (G, Turkey)           | Porto (G, Portugal)           |
| Tottenham (H, England)         | Real Madrid (H, Spain)        |

Technical explanations for 2016-2017 round of 16 draw by Michael Heselschwerdt (UEFA Head of Club Competitions):

> _The draw for the round of 16 of the 2016-17 UEFA Champions League involves all the clubs that qualified from the group stage. This round will be played according to the knockout system... The draw will take place in accordance with the following conditions: group winners will be drawn against the runners-up; clubs from the same association cannot be drawn against each other; group winners must be drawn against runners-up from different group; the runners-up play the first leg at home. Each of the eight pairings will be determined by drawing one of the runners-up first then one of the eligible group winners the eight-balls containing the names of the runners-up are placed in one ball while the balls containing the names of the opponents are picked from the eight different group winner spots and placed into another ball. the computer will indicate after drawing each runner-up which group winners can be drawn in accordance with the conditions explained should we have only one team eligible to be drawn against a particular runner-up there will be no draw and the two teams will be paired. This procedure is repeated for all the eight pairings._

Sources:

* [UEFA Champions League round of 16 draw](https://www.uefa.com/uefachampionsleague/season=2018/draws/round=2000882/index.html#/)
* [UEFA Champions League 2016-2017 round of 16 draw](https://www.youtube.com/watch?v=KhNapmFjM10)

## Imports, classes, constants, and functions

In [1]:
import itertools
import numpy as np
import networkx as nx
from IPython.display import display_html

In [2]:
class Team:
    """
    Team representation storing club information regarding:
        - short name
        - association/country to which the club belongs
        - group in the previous stage of the Champions League
    """
    def __init__(self, name, country, group):  # Constructor
        self.name = name
        self.country = country
        self.group = group

    def __repr__(self):  # String representation of instances
        return '{} ({}, {})'.format(self.name, self.group, self.country)

    def __hash__(self):  # Required for list.index working
        return hash((self.name, self.country, self.group))

    def __eq__(self, other):  # Required for list.index working
        try:
            return (self.name, self.country, self.group) == (other.name, other.country, other.group)
        except AttributeError:
            return NotImplemented

In [3]:
def is_valid_draw(fixtures):
    """
    A draw is valid if each game confronts teams belonging to different countries and different groups
    """
    return all([w.group != r.group and w.country != r.country for w, r in fixtures])

def filter_winners(runner_up, winners):
    return filter(lambda w: w.group != runner_up.group and w.country != runner_up.country, winners)

def remove_winners_leading_to_dead_ends(eligible_winners, WINNERS_POT, runner_up, RUNNERS_UP_POT):
    """
    From the list eligible_winners must be removed those clubs leading to dead ends in the draw.
    """
    remaining_runners = RUNNERS_UP_POT[:]
    remaining_runners.remove(runner_up)
    valid_winners = []
    for w in eligible_winners:
        remaining_winners = WINNERS_POT[:]
        remaining_winners.remove(w)
        if exist_maximum_matching(remaining_runners, remaining_winners):
            valid_winners.append(w)
    return valid_winners

def exist_maximum_matching(remaining_runners, remaining_winners):
    """
    A bipartite graph is built using remaining_runners clubs (first class) and 
    remaining_winners clubs (second class). For each club in the first class, eligible clubs
    from the second class are calculated, and for each of these pairs of nodes (1st class, 2nd class)
    an edge is built. If the maximum matching for this bipartite graph is exactly the sum of 
    the sizes of remaining_runners and remaining_winners, then there is no dead ends yet.
    """
    G = nx.Graph()
    size = len(remaining_runners)
    G.add_nodes_from(range(size), bipartite=0)
    G.add_nodes_from(range(size, 2*size), bipartite=1)
    for idx, r in enumerate(remaining_runners):
        for fw in filter_winners(r, remaining_winners):
            w_idx = remaining_winners.index(fw)
            G.add_edge(idx, w_idx + size)
    max_size = len(nx.algorithms.bipartite.maximum_matching(G))
    return max_size == 2 * size

def print_html(string):
    """
    Utility function to display HTML in a code cell
    """
    display_html(string, raw=True)

def build_html_table(runners_up, winners, probabilities):
    html = "<table>"
    html += "<tr><td>&nbsp;</td>"
    html += "<td><b>%s</b></td>" % ("</b></td><td><b>".join([x.name for x in runners_up]))
    html += "<td>CHECK</td></tr>"

    for w_idx in range(len(winners)):
        html += "<tr><td><b>%s</b></td>" % winners[w_idx].name
        html += "<td>%s</td>" % "</td><td>".join(["%.1f%%" % (x) for x in probabilities[w_idx,:]])
        html += "<td>%.1f%%</td></tr>" % sum(probabilities[w_idx,:])

    html += "<tr><td>CHECK</td>"
    html += "<td>%s</td><td>&nbsp;</td></tr>" % ("</td><td>".join(["%.1f%%" % (sum(probabilities[:,x])) \
                                                                   for x in range(len(runners_up))]))
    html += "</table>"
    return html

In [4]:
# Winners pot composition
WINNERS = [Team('Manchester United', 'England', 'A'), Team('PSG', 'France', 'B'),
           Team('Roma', 'Italy', 'C'),                Team('Barcelona', 'Spain', 'D'),
           Team('Liverpool', 'England', 'E'),         Team('Manchester City', 'England', 'F'),
           Team('Besiktas', 'Turkey', 'G'),           Team('Tottenham', 'England', 'H')]

# Runners-up pot composition
RUNNERS_UP = [Team('Basel', 'Switzerland', 'A'), Team('Bayer Munchen', 'Germany', 'B'),
              Team('Chelsea', 'England', 'C'),   Team('Juventus', 'Italy', 'D'),
              Team('Sevilla', 'Spain', 'E'),     Team('Shakhtar Donetsk', 'Ukraine', 'F'),
              Team('Porto', 'Portugal', 'G'),    Team('Real Madrid', 'Spain', 'H')]

## Draws calculation

In this case it is feasible to calculate all the possible outcomes of the draw. In a purely random draw, there would be $8! =40320$. Not all of them will be valid draws. There are two constraints about matches: a valid draw has no two teams from the same country and no two teams from the same group in the previous stage of the competition. After removing these draws from the list, there are still $4238$ valid draws. 

In [5]:
draws = [zip(WINNERS, x) for x in itertools.permutations(RUNNERS_UP)]
print("Total number of draws: %d" % len(draws))

valid_draws = filter(lambda x: is_valid_draw(x), draws)
print("Total number of valid draws: %d" % len(valid_draws))

Total number of draws: 40320
Total number of valid draws: 4238


## Probabilities calculation assuming an uniform distribution

As it was explained below, it is possible to compute all the possible draw outcomes. If we assume that all the outcomes have the same chance to be selected (technically we use an uniform distribution), the probability of a game will be the the ratio of favourable outcomes (valid draws containing that game) to the total number of possible outcomes (total number of valid draws). No Montecarlo simulation is necessary to complete this calculation. But this method ignores the procedure used by UEFA to complete the round of 16 draw in previous seasons of the competition.

In [6]:
total_events = float(len(valid_draws))  # total number of events
probabilities = np.full((len(WINNERS), len(RUNNERS_UP)), 0,  dtype=np.float32)  # store the probabilities

# Count how many times each pair of teams are matched in a valid draw
for draw in valid_draws:
    for winner, runner_up in draw:
        probabilities[WINNERS.index(winner), RUNNERS_UP.index(runner_up)] += 1

# Probability: the percentage of favourable outcomes to the total number of possible outcomes
probabilities = 100 * probabilities/total_events

In [7]:
# HTML output
html = build_html_table(RUNNERS_UP, WINNERS, probabilities)
print_html(html)

,Basel,Bayer Munchen,Chelsea,Juventus,Sevilla,Shakhtar Donetsk,Porto,Real Madrid,CHECK
Manchester United,0.0%,14.8%,0.0%,18.3%,18.3%,15.3%,14.8%,18.3%,100.0%
PSG,11.3%,0.0%,28.1%,12.8%,12.8%,11.3%,10.8%,12.8%,100.0%
Roma,15.8%,15.3%,0.0%,0.0%,18.9%,15.8%,15.3%,18.9%,100.0%
Barcelona,14.6%,13.5%,43.7%,0.0%,0.0%,14.6%,13.5%,0.0%,100.0%
Liverpool,15.8%,15.3%,0.0%,18.9%,0.0%,15.8%,15.3%,18.9%,100.0%
Manchester City,15.3%,14.8%,0.0%,18.3%,18.3%,0.0%,14.8%,18.3%,100.0%
Besiktas,11.3%,10.8%,28.1%,12.8%,12.8%,11.3%,0.0%,12.8%,100.0%
Tottenham,15.8%,15.3%,0.0%,18.9%,18.9%,15.8%,15.3%,0.0%,100.0%
CHECK,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,


## Probabilities estimated by a naïve Montecarlo simulation

Assuming that UEFA will use the same procedure as in the 2016/17 season, the pairings are selected in sequence by the following steps:

1. A club from the runners-up pot is selected.
2. A club from the winners pot that accomplised the constraints of the draw is selected.

The code below is simpler than it should be. In the actual draw, teams from winners pot are selected in order to avoid dead ends. We discard a draw when an dead end is reached and a new draw starts. It is questionable that this doesn't add a bias in the estimation. A more complex `filter_winners` would be better off.

In [8]:
simulations = 1000  # number of simulated draws
probabilities = np.full((len(WINNERS), len(RUNNERS_UP)), 0,  dtype=np.float32)  # store the probabilities

simulation = 0
while simulation < simulations:
    WINNERS_POT = WINNERS[:]
    RUNNERS_UP_POT = RUNNERS_UP[:]
    draw = np.full((len(WINNERS), len(RUNNERS_UP)), 0)  # array storing the results of a simulation
    failed = False  # if a simulated doesn't fulfill the constraints, restart the draw procedure
    #print("Simulation #%d" % (simulation+1))

    while len(RUNNERS_UP_POT) > 0:
        # First choose a team from runners-up pot
        runner_up = np.random.choice(RUNNERS_UP_POT)
        # Eligible teams from winners pot for that runner-up
        eligible_winners = filter_winners(runner_up, WINNERS_POT)
        #print("\tRunner-up: %s --> %s" % (runner_up.name, [x.name for x in eligible_winners]))

        if len(eligible_winners) > 0:
            # Draw a Team from winner pot
            winner = np.random.choice(eligible_winners)
            #print("\t\tWinner: %s" % winner.name)

            RUNNERS_UP_POT.remove(runner_up)  # remove the runner-up club from their pot
            WINNERS_POT.remove(winner)  # remove the winner club from their pot
            draw[WINNERS.index(winner), RUNNERS_UP.index(runner_up)] = 1
        else:
            #print("\tFailed")
            failed = True
            break
    if not failed:
        probabilities += draw
        simulation += 1  # correct simulated draw, process the next simulation
probabilities = 100 * probabilities / simulations

In [9]:
# HTML output
html = build_html_table(RUNNERS_UP, WINNERS, probabilities)
print_html(html)

,Basel,Bayer Munchen,Chelsea,Juventus,Sevilla,Shakhtar Donetsk,Porto,Real Madrid,CHECK
Manchester United,0.0%,15.1%,0.0%,17.8%,18.5%,16.5%,12.5%,19.6%,100.0%
PSG,10.2%,0.0%,29.6%,13.3%,12.7%,11.5%,10.1%,12.6%,100.0%
Roma,19.6%,13.2%,0.0%,0.0%,18.6%,14.2%,17.6%,16.8%,100.0%
Barcelona,13.9%,13.5%,43.8%,0.0%,0.0%,14.9%,13.9%,0.0%,100.0%
Liverpool,15.8%,16.7%,0.0%,18.9%,0.0%,15.1%,15.7%,17.8%,100.0%
Manchester City,16.0%,15.0%,0.0%,18.1%,16.8%,0.0%,14.9%,19.2%,100.0%
Besiktas,10.7%,11.1%,26.6%,11.3%,14.6%,11.7%,0.0%,14.0%,100.0%
Tottenham,13.8%,15.4%,0.0%,20.6%,18.8%,16.1%,15.3%,0.0%,100.0%
CHECK,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,


The result is consistent with the uniform distribution assumption. Definitely, a more complex `filter_winners` is mandatory.

## Probabilities estimated by a realistic Montecarlo simulation

To reproduce the very same steps of the draw in the Montecarlo simulation, dead ends must be avoided before they occur. This can be achieved by applying an additional filter to the candidates clubs from winners pot. The functions `remove_winners_leading_to_dead_ends` and `exist_maximum_matching` implement this by means of [bipartite graphs](https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.bipartite.html) and [maximum matching algorithm](https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.bipartite.html?highlight=bipartite#module-networkx.algorithms.bipartite.matching) from `networkx` package.

In [10]:
# A simple execution to check that everything works smoothly
WINNERS_POT = WINNERS[:]
RUNNERS_UP_POT = RUNNERS_UP[:]

print("Draw simulation...\n")
while len(RUNNERS_UP_POT) > 0:
    runner_up = np.random.choice(RUNNERS_UP_POT)
    eligible_winners = filter_winners(runner_up, WINNERS_POT)
    print("Runner-up: %s" % runner_up.name)
    print("\tValid winners: %s" % (', '.join([x.name for x in eligible_winners])))
    eligible_winners = remove_winners_leading_to_dead_ends(eligible_winners, WINNERS_POT, 
                                                           runner_up, RUNNERS_UP_POT)
    print("\tEligible winners --> %s" % (', '.join([x.name for x in eligible_winners])))

    if len(eligible_winners) > 0:
        # Draw a Team from winner pot
        winner = np.random.choice(eligible_winners)
        print("\tWinner drawn: %s" % winner.name)

        RUNNERS_UP_POT.remove(runner_up)  # remove the runner-up club from their pot
        WINNERS_POT.remove(winner)  # remove the winner club from their pot
        print("%s vs %s\n" % (runner_up.name, winner.name))
    else:
        print("\tFailed")
        failed = True
        break

Draw simulation...

Runner-up: Bayer Munchen
	Valid winners: Manchester United, Roma, Barcelona, Liverpool, Manchester City, Besiktas, Tottenham
	Eligible winners --> Manchester United, Roma, Barcelona, Liverpool, Manchester City, Besiktas, Tottenham
	Winner drawn: Barcelona
Bayer Munchen vs Barcelona

Runner-up: Juventus
	Valid winners: Manchester United, PSG, Liverpool, Manchester City, Besiktas, Tottenham
	Eligible winners --> Manchester United, PSG, Liverpool, Manchester City, Besiktas, Tottenham
	Winner drawn: Manchester City
Juventus vs Manchester City

Runner-up: Real Madrid
	Valid winners: Manchester United, PSG, Roma, Liverpool, Besiktas
	Eligible winners --> Manchester United, PSG, Roma, Liverpool, Besiktas
	Winner drawn: Roma
Real Madrid vs Roma

Runner-up: Basel
	Valid winners: PSG, Liverpool, Besiktas, Tottenham
	Eligible winners --> PSG, Liverpool, Besiktas, Tottenham
	Winner drawn: Tottenham
Basel vs Tottenham

Runner-up: Chelsea
	Valid winners: PSG, Besiktas
	Eligible w

In [11]:
simulations = 1000  # number of simulated draws
probabilities = np.full((len(WINNERS), len(RUNNERS_UP)), 0,  dtype=np.float32)  # store the probabilities

simulation = 0
while simulation < simulations:
    WINNERS_POT = WINNERS[:]
    RUNNERS_UP_POT = RUNNERS_UP[:]

    while len(RUNNERS_UP_POT) > 0:
        runner_up = np.random.choice(RUNNERS_UP_POT)
        eligible_winners = filter_winners(runner_up, WINNERS_POT)
        #print("\tRunner-up: %s --> %s" % (runner_up.name, [x.name for x in eligible_winners]))
        eligible_winners = remove_winners_leading_to_dead_ends(eligible_winners, WINNERS_POT, 
                                                               runner_up, RUNNERS_UP_POT)
        #print("\t\tEffective candidates --> %s" % ([x.name for x in eligible_winners]))

        # Draw a Team from winner pot
        winner = np.random.choice(eligible_winners)
        #print("\t\tWinner: %s" % winner.name)

        RUNNERS_UP_POT.remove(runner_up)  # remove the runner-up club from their pot
        WINNERS_POT.remove(winner)  # remove the winner club from their pot
        probabilities[WINNERS.index(winner), RUNNERS_UP.index(runner_up)] += 1
    simulation += 1

probabilities = 100 * probabilities / simulations

In [12]:
# HTML output
html = build_html_table(RUNNERS_UP, WINNERS, probabilities)
print_html(html)

,Basel,Bayer Munchen,Chelsea,Juventus,Sevilla,Shakhtar Donetsk,Porto,Real Madrid,CHECK
Manchester United,0.0%,15.2%,0.0%,15.3%,17.8%,17.4%,15.3%,19.0%,100.0%
PSG,11.6%,0.0%,30.8%,12.3%,13.8%,10.5%,8.4%,12.6%,100.0%
Roma,17.8%,16.1%,0.0%,0.0%,17.1%,16.2%,15.6%,17.2%,100.0%
Barcelona,15.3%,15.6%,39.9%,0.0%,0.0%,13.9%,15.3%,0.0%,100.0%
Liverpool,16.3%,15.4%,0.0%,19.9%,0.0%,15.4%,15.8%,17.2%,100.0%
Manchester City,14.2%,12.8%,0.0%,18.2%,19.4%,0.0%,15.6%,19.8%,100.0%
Besiktas,8.6%,10.3%,29.3%,13.0%,13.7%,10.9%,0.0%,14.2%,100.0%
Tottenham,16.2%,14.6%,0.0%,21.3%,18.2%,15.7%,14.0%,0.0%,100.0%
CHECK,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,


There are some minor changes in the estimations. As Chelsea is the most constrained club in the draw, their probabilities are the only ones that change significantly with respect to the naïve Montecarlo simulation.